In [1]:
from dolfinx import default_scalar_type
from dolfinx.fem import (Constant, dirichletbc, Function, FunctionSpace, assemble_scalar,
                         form, locate_dofs_geometrical, locate_dofs_topological)
from dolfinx.fem.petsc import LinearProblem
from dolfinx.io import XDMFFile, gmshio
from dolfinx.mesh import create_unit_square, locate_entities
from dolfinx.plot import vtk_mesh


from ufl import (SpatialCoordinate, TestFunction, TrialFunction, dx, grad, inner)
from mpi4py import MPI
import meshio
import gmsh
import pyvista
import numpy as np
import ufl
from petsc4py import PETSc

from dolfinx.fem.petsc import assemble_vector, assemble_matrix, create_vector, apply_lifting, set_bc
from dolfinx import *
from dolfinx import fem, mesh, plot ,io 



In [2]:
# Define temporal parameters
t = 0    # Start time
T = 2.0  # Final time
num_steos = 50
dt = T / num_steos  # time step size

# coefficient of friction
k = 100
u_in = 20    # inlet temperature
u_out = -20  # outlet temperature


In [3]:
# Define mesh
domain, cell_markers, facet_markers = gmshio.read_from_msh("helix.msh", MPI.COMM_WORLD, gdim=3)


Info    : Reading 'helix.msh'...
Info    : 7982 nodes
Info    : 51264 elements
Info    : Done reading 'helix.msh'


In [4]:
V = fem.FunctionSpace(domain, ("Lagrange", 1))

In [5]:
# Create initial condition

dofs_L = locate_dofs_geometrical(V, lambda x: np.isclose(x[0], 0))
u_L = Function(V)
u_L.interpolate(lambda x: -20 + x[1])
bc_L = dirichletbc(u_L, dofs_L)


dofs_R = locate_dofs_geometrical(V, lambda x: np.isclose(x[0], 1))
u_R = Function(V)
u_R.interpolate(lambda x: 20 + x[1])
bc_R = dirichletbc(u_R, dofs_R)

bcs = [bc_R, bc_L]

u_n = fem.Function(V)

# Time-dependent output

In [6]:
xdmf = io.XDMFFile(domain.comm, "helix.xdmf", "w")
xdmf.write_mesh(domain)

# Define solution variable, and interpolate initial solution for visualization in Paraview
uh = fem.Function(V)
uh.name = "uh"
#uh.interpolate(initial_condition)
xdmf.write_function(uh, t)

# Variational problem and solver

In [7]:
u, v = ufl.TrialFunction(V), ufl.TestFunction(V)
f = fem.Constant(domain, PETSc.ScalarType(0))
a = u * v * ufl.dx + dt * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = (u_n + dt * f) * v * ufl.dx

# Preparing linear algerba structures for time dependent problems

In [8]:
bilinear_form = fem.form(a)
linear_form = fem.form(L)

In [9]:
A = assemble_matrix(bilinear_form, bcs)
A.assemble()
b = create_vector(linear_form)

# Using petsc4py to create a linear solver

In [10]:
solver = PETSc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

# Visualization of time dependent problem using pyvista

In [11]:
import matplotlib as mpl
pyvista.start_xvfb()

grid = pyvista.UnstructuredGrid(*plot.vtk_mesh(V))

plotter = pyvista.Plotter()
plotter.open_gif("u_time.gif", fps=10)

grid.point_data["uh"] = uh.x.array
warped = grid.warp_by_scalar("uh", factor=1)

viridis = mpl.colormaps.get_cmap("viridis").resampled(25)
sargs = dict(title_font_size=25, label_font_size=20, fmt="%.2e", color="black",
             position_x=0.1, position_y=0.8, width=0.8, height=0.1)

renderer = plotter.add_mesh(warped, show_edges=True, lighting=False,
                            cmap=viridis, scalar_bar_args=sargs,
                            clim=[0, max(uh.x.array)])

AttributeError: 'ColormapRegistry' object has no attribute 'get_cmap'